# **Deforum Stable Diffusion v0.7 With Automated Music Sync**
[Stable Diffusion](https://github.com/CompVis/stable-diffusion) by Robin Rombach, Andreas Blattmann, Dominik Lorenz, Patrick Esser, Björn Ommer and the [Stability.ai](https://stability.ai/) Team. [K Diffusion](https://github.com/crowsonkb/k-diffusion) by [Katherine Crowson](https://twitter.com/RiversHaveWings). Notebook by [deforum](https://discord.gg/upmXXsrwZc)

[Quick Guide](https://docs.google.com/document/d/1RrQv7FntzOuLg4ohjRZPVL7iptIyBhwwbcEYEW2OfcI/edit?usp=sharing) to Deforum v0.7

In [ ]:
#@markdown **NVIDIA GPU**
import subprocess, os, sys
sub_p_res = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total,memory.free', '--format=csv,noheader'], stdout=subprocess.PIPE).stdout.decode('utf-8')
print(f"{sub_p_res[:-1]}")

# Setup

In [ ]:
#@markdown **Environment Setup**
import subprocess, time, gc, os, sys

def setup_environment():
    start_time = time.time()
    print_subprocess = False
    use_xformers_for_colab = True
    try:
        ipy = get_ipython()
    except:
        ipy = 'could not get_ipython'
    if 'google.colab' in str(ipy):
        print("..setting up environment")
        
        all_process = [
            ['pip', 'install', 'torch==1.12.1+cu113', 'torchvision==0.13.1+cu113', '--extra-index-url', 'https://download.pytorch.org/whl/cu113'],
            ['pip', 'install', 'omegaconf==2.2.3', 'einops==0.4.1', 'pytorch-lightning==1.7.4', 'torchmetrics==0.9.3', 'torchtext==0.13.1', 'transformers==4.21.2', 'safetensors', 'kornia==0.6.7'],
            ['git', 'clone', 'https://github.com/deforum-art/deforum-stable-diffusion'],
            ['pip', 'install', 'accelerate', 'ftfy', 'jsonmerge', 'matplotlib', 'resize-right', 'timm', 'torchdiffeq','scikit-learn','torchsde','open-clip-torch','numpngw'],
        ]
        for process in all_process:
            running = subprocess.run(process,stdout=subprocess.PIPE).stdout.decode('utf-8')
            if print_subprocess:
                print(running)
        with open('deforum-stable-diffusion/src/k_diffusion/__init__.py', 'w') as f:
            f.write('')
        sys.path.extend([
            'deforum-stable-diffusion/',
            'deforum-stable-diffusion/src',
        ])
        if use_xformers_for_colab:

            print("..installing xformers")

            all_process = [['pip', 'install', 'triton==2.0.0.dev20220701']]
            for process in all_process:
                running = subprocess.run(process,stdout=subprocess.PIPE).stdout.decode('utf-8')
                if print_subprocess:
                    print(running)

            v_card_name = subprocess.run(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], stdout=subprocess.PIPE).stdout.decode('utf-8')
            if 't4' in v_card_name.lower():
                name_to_download = 'T4'
            elif 'v100' in v_card_name.lower():
                name_to_download = 'V100'
            elif 'a100' in v_card_name.lower():
                name_to_download = 'A100'
            elif 'p100' in v_card_name.lower():
                name_to_download = 'P100'
            elif 'a4000' in v_card_name.lower():
                name_to_download = 'Non-Colab/Paperspace/A4000'
            elif 'p5000' in v_card_name.lower():
                name_to_download = 'Non-Colab/Paperspace/P5000'
            elif 'quadro m4000' in v_card_name.lower():
                name_to_download = 'Non-Colab/Paperspace/Quadro M4000'
            elif 'rtx 4000' in v_card_name.lower():
                name_to_download = 'Non-Colab/Paperspace/RTX 4000'
            elif 'rtx 5000' in v_card_name.lower():
                name_to_download = 'Non-Colab/Paperspace/RTX 5000'
            else:
                print(v_card_name + ' is currently not supported with xformers flash attention in deforum!')

            if 'Non-Colab' in name_to_download:
                x_ver = 'xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl'
            else:
                x_ver = 'xformers-0.0.13.dev0-py3-none-any.whl'

            x_link = 'https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/' + name_to_download + '/' + x_ver

            all_process = [
                ['wget', '--no-verbose', '--no-clobber', x_link],
                ['pip', 'install', x_ver],
            ]

            for process in all_process:
                running = subprocess.run(process,stdout=subprocess.PIPE).stdout.decode('utf-8')
                if print_subprocess:
                    print(running)
    else:
        sys.path.extend([
            'src'
        ])
    end_time = time.time()
    print(f"..environment set up in {end_time-start_time:.0f} seconds")
    return

setup_environment()

import torch
import random
import clip
from IPython import display
from types import SimpleNamespace
from helpers.save_images import get_output_folder
from helpers.settings import load_args
from helpers.render import render_animation, render_input_video, render_image_batch, render_interpolation
from helpers.model_load import make_linear_decode, load_model, get_model_output_paths
from helpers.aesthetics import load_aesthetics_model

In [ ]:
#@markdown **Path Setup**

def Root():
    models_path = "models" #@param {type:"string"}
    configs_path = "configs" #@param {type:"string"}
    output_path = "outputs" #@param {type:"string"}
    mount_google_drive = False #@param {type:"boolean"}
    models_path_gdrive = "/content/drive/MyDrive/AI/models" #@param {type:"string"}
    output_path_gdrive = "/content/drive/MyDrive/AI/StableDiffusion" #@param {type:"string"}

    #@markdown **Model Setup**
    model_config = "v1-inference.yaml" #@param ["custom","v2-inference.yaml","v2-inference-v.yaml","v1-inference.yaml"]
    model_checkpoint =  "sd-v1-4.ckpt" #@param ["custom","v2-1_768-ema-pruned.ckpt","v2-1_512-ema-pruned.ckpt","768-v-ema.ckpt","512-base-ema.ckpt","v1-5-pruned.ckpt","v1-5-pruned-emaonly.ckpt","sd-v1-4-full-ema.ckpt","sd-v1-4.ckpt","sd-v1-3-full-ema.ckpt","sd-v1-3.ckpt","sd-v1-2-full-ema.ckpt","sd-v1-2.ckpt","sd-v1-1-full-ema.ckpt","sd-v1-1.ckpt", "robo-diffusion-v1.ckpt","wd-v1-3-float16.ckpt"]
    custom_config_path = "" #@param {type:"string"}
    custom_checkpoint_path = "" #@param {type:"string"}
    return locals()

root = Root()
root = SimpleNamespace(**root)

root.models_path, root.output_path = get_model_output_paths(root)
root.model, root.device = load_model(root, load_on_run_all=True, check_sha256=True)

# Audio Sync settings
You can control animation settings like zoom/angle etc based on a music aplitudes. To do so, we need to analyze a music track and generate a keyframe string. Fortunatelly, there is a colab notebook that automates the whole process. You can use this [Notebook](https://colab.research.google.com/github/fzantalis/colab_collection/blob/master/Audio_Keyframe_Generator_For_Deforum_Stable_Diffusion.ipynb) and
copy/paste the keyframe string to the 'zoom' setting below, or play with the function and generate keyframes for any other setting

# Settings

In [ ]:
def DeforumAnimArgs():

    #@markdown ####**Animation:**
    animation_mode = '2D' #@param ['None', '2D', '3D', 'Video Input', 'Interpolation'] {type:'string'}
    max_frames = 719 #@param {type:"number"}
    border = 'replicate' #@param ['wrap', 'replicate'] {type:'string'}

    #@markdown ####**Motion Parameters:**
    angle = "0: (-2), 50: (0), 100: (-2), 150: (1), 200: (1), 250: (-1), 300: (-2), 350: (-2), 400: (1), 450: (-2), 500: (0), 550: (-2), 600: (-1), 650: (-2), 700: (-1), "#@param {type:"string"}
    zoom = "0: (1.07), 1: (1.07), 2: (1.07), 3: (1.07), 4: (1.07), 5: (1.07), 6: (1.07), 7: (1.07), 8: (1.07), 9: (1.07), 10: (1.07), 11: (1.07), 12: (1.07), 13: (1.07), 14: (1.09), 15: (1.07), 16: (1.07), 17: (1.07), 18: (1.07), 19: (1.07), 20: (1.07), 21: (1.07), 22: (1.07), 23: (1.07), 24: (1.07), 25: (1.07), 26: (1.07), 27: (1.07), 28: (1.07), 29: (1.07), 30: (1.07), 31: (1.37), 32: (1.07), 33: (1.07), 34: (1.07), 35: (1.07), 36: (1.07), 37: (1.07), 38: (1.07), 39: (1.07), 40: (1.07), 41: (1.07), 42: (1.07), 43: (1.07), 44: (1.07), 45: (1.07), 46: (1.07), 47: (1.11), 48: (1.11), 49: (1.07), 50: (1.07), 51: (1.07), 52: (1.07), 53: (1.07), 54: (1.07), 55: (1.07), 56: (1.07), 57: (1.07), 58: (1.14), 59: (1.38), 60: (1.07), 61: (1.07), 62: (1.07), 63: (1.07), 64: (1.07), 65: (1.07), 66: (1.07), 67: (1.07), 68: (1.07), 69: (1.07), 70: (1.07), 71: (1.07), 72: (1.07), 73: (1.07), 74: (1.07), 75: (1.27), 76: (1.07), 77: (1.07), 78: (1.07), 79: (1.07), 80: (1.07), 81: (1.07), 82: (1.07), 83: (1.07), 84: (1.07), 85: (1.07), 86: (1.07), 87: (1.07), 88: (1.07), 89: (1.07), 90: (1.07), 91: (1.07), 92: (1.33), 93: (1.07), 94: (1.07), 95: (1.07), 96: (1.07), 97: (1.07), 98: (1.07), 99: (1.07), 100: (1.07), 101: (1.07), 102: (1.07), 103: (1.33), 104: (1.07), 105: (1.07), 106: (1.07), 107: (1.07), 108: (1.07), 109: (1.07), 110: (1.07), 111: (1.07), 112: (1.07), 113: (1.07), 114: (1.07), 115: (1.07), 116: (1.07), 117: (1.07), 118: (1.07), 119: (1.14), 120: (1.21), 121: (1.07), 122: (1.07), 123: (1.07), 124: (1.07), 125: (1.07), 126: (1.07), 127: (1.07), 128: (1.07), 129: (1.07), 130: (1.07), 131: (1.07), 132: (1.07), 133: (1.07), 134: (1.07), 135: (1.07), 136: (1.30), 137: (1.07), 138: (1.07), 139: (1.07), 140: (1.07), 141: (1.07), 142: (1.07), 143: (1.07), 144: (1.07), 145: (1.07), 146: (1.07), 147: (1.34), 148: (1.07), 149: (1.07), 150: (1.07), 151: (1.07), 152: (1.07), 153: (1.07), 154: (1.07), 155: (1.07), 156: (1.07), 157: (1.07), 158: (1.07), 159: (1.07), 160: (1.07), 161: (1.07), 162: (1.07), 163: (1.07), 164: (1.07), 165: (1.07), 166: (1.07), 167: (1.07), 168: (1.07), 169: (1.07), 170: (1.07), 171: (1.07), 172: (1.07), 173: (1.07), 174: (1.07), 175: (1.07), 176: (1.07), 177: (1.07), 178: (1.07), 179: (1.07), 180: (1.13), 181: (1.16), 182: (1.07), 183: (1.07), 184: (1.07), 185: (1.07), 186: (1.07), 187: (1.07), 188: (1.07), 189: (1.07), 190: (1.07), 191: (1.09), 192: (1.20), 193: (1.07), 194: (1.07), 195: (1.07), 196: (1.07), 197: (1.07), 198: (1.07), 199: (1.07), 200: (1.07), 201: (1.07), 202: (1.07), 203: (1.07), 204: (1.07), 205: (1.07), 206: (1.07), 207: (1.07), 208: (1.26), 209: (1.07), 210: (1.07), 211: (1.07), 212: (1.07), 213: (1.07), 214: (1.07), 215: (1.07), 216: (1.07), 217: (1.07), 218: (1.07), 219: (1.07), 220: (1.07), 221: (1.07), 222: (1.07), 223: (1.07), 224: (1.07), 225: (1.21), 226: (1.07), 227: (1.07), 228: (1.07), 229: (1.07), 230: (1.07), 231: (1.07), 232: (1.07), 233: (1.07), 234: (1.07), 235: (1.07), 236: (1.65), 237: (1.07), 238: (1.07), 239: (1.07), 240: (1.07), 241: (1.07), 242: (1.07), 243: (1.07), 244: (1.07), 245: (1.07), 246: (1.07), 247: (1.07), 248: (1.07), 249: (1.07), 250: (1.07), 251: (1.07), 252: (1.09), 253: (1.16), 254: (1.07), 255: (1.07), 256: (1.07), 257: (1.07), 258: (1.07), 259: (1.07), 260: (1.07), 261: (1.07), 262: (1.07), 263: (1.07), 264: (1.07), 265: (1.07), 266: (1.07), 267: (1.07), 268: (1.07), 269: (1.28), 270: (1.08), 271: (1.07), 272: (1.07), 273: (1.07), 274: (1.07), 275: (1.07), 276: (1.07), 277: (1.07), 278: (1.07), 279: (1.07), 280: (1.26), 281: (1.08), 282: (1.07), 283: (1.07), 284: (1.07), 285: (1.07), 286: (1.07), 287: (1.07), 288: (1.07), 289: (1.07), 290: (1.07), 291: (1.07), 292: (1.07), 293: (1.07), 294: (1.07), 295: (1.07), 296: (1.07), 297: (1.39), 298: (1.07), 299: (1.07), 300: (1.07), 301: (1.07), 302: (1.07), 303: (1.07), 304: (1.07), 305: (1.07), 306: (1.07), 307: (1.07), 308: (1.07), 309: (1.07), 310: (1.07), 311: (1.07), 312: (1.07), 313: (1.10), 314: (1.10), 315: (1.07), 316: (1.07), 317: (1.07), 318: (1.07), 319: (1.07), 320: (1.07), 321: (1.07), 322: (1.07), 323: (1.07), 324: (1.12), 325: (1.14), 326: (1.07), 327: (1.07), 328: (1.07), 329: (1.07), 330: (1.07), 331: (1.07), 332: (1.07), 333: (1.07), 334: (1.07), 335: (1.07), 336: (1.07), 337: (1.07), 338: (1.07), 339: (1.07), 340: (1.07), 341: (1.08), 342: (1.07), 343: (1.07), 344: (1.07), 345: (1.07), 346: (1.07), 347: (1.07), 348: (1.07), 349: (1.07), 350: (1.07), 351: (1.07), 352: (1.07), 353: (1.07), 354: (1.07), 355: (1.07), 356: (1.07), 357: (1.07), 358: (1.17), 359: (1.07), 360: (1.07), 361: (1.07), 362: (1.07), 363: (1.07), 364: (1.07), 365: (1.07), 366: (1.07), 367: (1.07), 368: (1.07), 369: (1.28), 370: (1.07), 371: (1.07), 372: (1.07), 373: (1.07), 374: (1.07), 375: (1.07), 376: (1.07), 377: (1.07), 378: (1.07), 379: (1.07), 380: (1.07), 381: (1.07), 382: (1.07), 383: (1.07), 384: (1.07), 385: (1.08), 386: (1.11), 387: (1.07), 388: (1.07), 389: (1.07), 390: (1.07), 391: (1.07), 392: (1.07), 393: (1.07), 394: (1.07), 395: (1.07), 396: (1.07), 397: (1.07), 398: (1.07), 399: (1.07), 400: (1.07), 401: (1.07), 402: (1.14), 403: (1.07), 404: (1.07), 405: (1.07), 406: (1.07), 407: (1.07), 408: (1.07), 409: (1.07), 410: (1.07), 411: (1.07), 412: (1.07), 413: (1.22), 414: (1.08), 415: (1.07), 416: (1.07), 417: (1.07), 418: (1.07), 419: (1.07), 420: (1.07), 421: (1.07), 422: (1.07), 423: (1.07), 424: (1.07), 425: (1.07), 426: (1.07), 427: (1.07), 428: (1.07), 429: (1.07), 430: (2.01), 431: (1.07), 432: (1.07), 433: (1.07), 434: (1.07), 435: (1.07), 436: (1.07), 437: (1.07), 438: (1.07), 439: (1.07), 440: (1.07), 441: (1.07), 442: (1.07), 443: (1.07), 444: (1.07), 445: (1.07), 446: (1.09), 447: (1.15), 448: (1.07), 449: (1.07), 450: (1.07), 451: (1.07), 452: (1.07), 453: (1.07), 454: (1.07), 455: (1.07), 456: (1.07), 457: (1.08), 458: (1.25), 459: (1.07), 460: (1.07), 461: (1.07), 462: (1.07), 463: (1.07), 464: (1.07), 465: (1.07), 466: (1.07), 467: (1.07), 468: (1.07), 469: (1.07), 470: (1.07), 471: (1.07), 472: (1.07), 473: (1.07), 474: (1.27), 475: (1.08), 476: (1.07), 477: (1.07), 478: (1.07), 479: (1.07), 480: (1.07), 481: (1.07), 482: (1.07), 483: (1.07), 484: (1.07), 485: (1.07), 486: (1.07), 487: (1.07), 488: (1.07), 489: (1.07), 490: (1.07), 491: (1.29), 492: (1.07), 493: (1.07), 494: (1.07), 495: (1.07), 496: (1.07), 497: (1.07), 498: (1.07), 499: (1.07), 500: (1.07), 501: (1.07), 502: (1.28), 503: (1.07), 504: (1.07), 505: (1.07), 506: (1.07), 507: (1.07), 508: (1.07), 509: (1.07), 510: (1.07), 511: (1.07), 512: (1.07), 513: (1.07), 514: (1.07), 515: (1.07), 516: (1.07), 517: (1.07), 518: (1.07), 519: (1.07), 520: (1.07), 521: (1.07), 522: (1.07), 523: (1.07), 524: (1.07), 525: (1.07), 526: (1.07), 527: (1.07), 528: (1.07), 529: (1.07), 530: (1.07), 531: (1.07), 532: (1.07), 533: (1.07), 534: (1.07), 535: (1.22), 536: (1.09), 537: (1.07), 538: (1.07), 539: (1.07), 540: (1.07), 541: (1.07), 542: (1.07), 543: (1.07), 544: (1.07), 545: (1.07), 546: (1.22), 547: (1.09), 548: (1.07), 549: (1.07), 550: (1.07), 551: (1.07), 552: (1.07), 553: (1.07), 554: (1.07), 555: (1.07), 556: (1.07), 557: (1.07), 558: (1.07), 559: (1.07), 560: (1.07), 561: (1.07), 562: (1.07), 563: (2.07), 564: (1.07), 565: (1.07), 566: (1.07), 567: (1.07), 568: (1.07), 569: (1.07), 570: (1.07), 571: (1.07), 572: (1.07), 573: (1.07), 574: (1.07), 575: (1.07), 576: (1.07), 577: (1.07), 578: (1.07), 579: (1.07), 580: (1.10), 581: (1.07), 582: (1.07), 583: (1.07), 584: (1.07), 585: (1.07), 586: (1.07), 587: (1.07), 588: (1.07), 589: (1.07), 590: (1.08), 591: (1.31), 592: (1.07), 593: (1.07), 594: (1.07), 595: (1.07), 596: (1.07), 597: (1.07), 598: (1.07), 599: (1.07), 600: (1.07), 601: (1.07), 602: (1.07), 603: (1.07), 604: (1.07), 605: (1.07), 606: (1.07), 607: (1.36), 608: (1.09), 609: (1.07), 610: (1.07), 611: (1.07), 612: (1.07), 613: (1.07), 614: (1.07), 615: (1.07), 616: (1.07), 617: (1.07), 618: (1.07), 619: (1.07), 620: (1.07), 621: (1.07), 622: (1.07), 623: (1.07), 624: (1.20), 625: (1.07), 626: (1.07), 627: (1.07), 628: (1.07), 629: (1.07), 630: (1.07), 631: (1.07), 632: (1.07), 633: (1.07), 634: (1.07), 635: (1.41), 636: (1.07), 637: (1.07), 638: (1.07), 639: (1.07), 640: (1.07), 641: (1.07), 642: (1.07), 643: (1.07), 644: (1.07), 645: (1.07), 646: (1.07), 647: (1.07), 648: (1.07), 649: (1.07), 650: (1.07), 651: (1.07), 652: (1.15), 653: (1.07), 654: (1.07), 655: (1.07), 656: (1.07), 657: (1.07), 658: (1.07), 659: (1.07), 660: (1.07), 661: (1.07), 662: (1.07), 663: (1.07), 664: (1.07), 665: (1.07), 666: (1.07), 667: (1.07), 668: (1.07), 669: (1.07), 670: (1.07), 671: (1.07), 672: (1.07), 673: (1.07), 674: (1.07), 675: (1.07), 676: (1.07), 677: (1.07), 678: (1.07), 679: (1.07), 680: (1.07), 681: (1.07), 682: (1.07), 683: (1.07), 684: (1.07), 685: (1.07), 686: (1.07), 687: (1.07), 688: (1.07), 689: (1.07), 690: (1.07), 691: (1.07), 692: (1.07), 693: (1.07), 694: (1.07), 695: (1.07), 696: (1.07), 697: (1.07), 698: (1.07), 699: (1.07), 700: (1.07), 701: (1.07), 702: (1.07), 703: (1.07), 704: (1.07), 705: (1.07), 706: (1.07), 707: (1.07), 708: (1.07), 709: (1.07), 710: (1.07), 711: (1.07), 712: (1.07), 713: (1.07), 714: (1.07), 715: (1.07), 716: (1.07), 717: (1.07), 718: (1.07), 719: (1.07), "#@param {type:"string"}
    translation_x = "0:(10*sin(2*3.14*t/10))"#@param {type:"string"}
    translation_y = "0:(0)"#@param {type:"string"}
    translation_z = "0:(10)"#@param {type:"string"}
    rotation_3d_x = "0:(0)"#@param {type:"string"}
    rotation_3d_y = "0:(0)"#@param {type:"string"}
    rotation_3d_z = "0:(0)"#@param {type:"string"}
    flip_2d_perspective = False #@param {type:"boolean"}
    perspective_flip_theta = "0:(0)"#@param {type:"string"}
    perspective_flip_phi = "0:(t%15)"#@param {type:"string"}
    perspective_flip_gamma = "0:(0)"#@param {type:"string"}
    perspective_flip_fv = "0:(53)"#@param {type:"string"}
    noise_schedule = "0: (0.02)"#@param {type:"string"}
    strength_schedule = "0: (0.50)"#@param {type:"string"}
    contrast_schedule = "0: (1.0)"#@param {type:"string"}
    hybrid_video_comp_alpha_schedule = "0:(1)" #@param {type:"string"}
    hybrid_video_comp_mask_blend_alpha_schedule = "0:(0.5)" #@param {type:"string"}
    hybrid_video_comp_mask_contrast_schedule = "0:(1)" #@param {type:"string"}
    hybrid_video_comp_mask_auto_contrast_cutoff_high_schedule =  "0:(100)" #@param {type:"string"}
    hybrid_video_comp_mask_auto_contrast_cutoff_low_schedule =  "0:(0)" #@param {type:"string"}

    #@markdown ####**Unsharp mask (anti-blur) Parameters:**
    kernel_schedule = "0: (5)"#@param {type:"string"}
    sigma_schedule = "0: (1.0)"#@param {type:"string"}
    amount_schedule = "0: (0.2)"#@param {type:"string"}
    threshold_schedule = "0: (0.0)"#@param {type:"string"}

    #@markdown ####**Coherence:**
    color_coherence = 'Match Frame 0 LAB' #@param ['None', 'Match Frame 0 HSV', 'Match Frame 0 LAB', 'Match Frame 0 RGB', 'Video Input'] {type:'string'}
    color_coherence_video_every_N_frames = 1 #@param {type:"integer"}
    diffusion_cadence = '3' #@param ['1','2','3','4','5','6','7','8'] {type:'string'}

    #@markdown ####**3D Depth Warping:**
    use_depth_warping = True #@param {type:"boolean"}
    midas_weight = 0.3#@param {type:"number"}
    near_plane = 200
    far_plane = 10000
    fov = 40#@param {type:"number"}
    padding_mode = 'border'#@param ['border', 'reflection', 'zeros'] {type:'string'}
    sampling_mode = 'bicubic'#@param ['bicubic', 'bilinear', 'nearest'] {type:'string'}
    save_depth_maps = False #@param {type:"boolean"}

    #@markdown ####**Video Input:**
    video_init_path ='/content/video_in.mp4'#@param {type:"string"}
    extract_nth_frame = 1#@param {type:"number"}
    overwrite_extracted_frames = True #@param {type:"boolean"}
    use_mask_video = False #@param {type:"boolean"}
    video_mask_path ='/content/video_in.mp4'#@param {type:"string"}

    #@markdown ####**Hybrid Video for 2D/3D Animation Mode:**
    hybrid_video_generate_inputframes = False #@param {type:"boolean"}
    hybrid_video_use_first_frame_as_init_image = True #@param {type:"boolean"}
    hybrid_video_motion = "None" #@param ['None','Optical Flow','Perspective','Affine']
    hybrid_video_flow_method = "Farneback" #@param ['Farneback','DenseRLOF','SF']
    hybrid_video_composite = False #@param {type:"boolean"}
    hybrid_video_comp_mask_type = "None" #@param ['None', 'Depth', 'Video Depth', 'Blend', 'Difference']
    hybrid_video_comp_mask_inverse = False #@param {type:"boolean"}
    hybrid_video_comp_mask_equalize = "None" #@param  ['None','Before','After','Both']
    hybrid_video_comp_mask_auto_contrast = False #@param {type:"boolean"}
    hybrid_video_comp_save_extra_frames = False #@param {type:"boolean"}
    hybrid_video_use_video_as_mse_image = False #@param {type:"boolean"}

    #@markdown ####**Interpolation:**
    interpolate_key_frames = False #@param {type:"boolean"}
    interpolate_x_frames = 4 #@param {type:"number"}
    
    #@markdown ####**Resume Animation:**
    resume_from_timestring = False #@param {type:"boolean"}
    resume_timestring = "20220829210106" #@param {type:"string"}

    return locals()

In [ ]:
prompts = [
    "a beautiful lake by Asher Brown Durand, trending on Artstation", # the first prompt I want
    "a beautiful portrait of a woman by Artgerm, trending on Artstation", # the second prompt I want
    #"this prompt I don't want it I commented it out",
    #"a nousr robot, trending on Artstation", # use "nousr robot" with the robot diffusion model (see model_checkpoint setting)
    #"touhou 1girl komeiji_koishi portrait, green hair", # waifu diffusion prompts can use danbooru tag groups (see model_checkpoint)
    #"this prompt has weights if prompt weighting enabled:2 can also do negative:-2", # (see prompt_weighting)
]

animation_prompts = {
    0: "a grayscale painting of void post mortem, digital art, hd, trending on artstation",
    200: "a grayscale painting of a gray prison coridors and bars, hd, trending on artstation",
    400: "a grayscale painting beautiful lonely girl waving goodbye, trending on artstation",
    600: "a grayscale painting of the feeling of loneliness, trending on Artstation",
}

In [ ]:
#@markdown **Load Settings**
override_settings_with_file = False #@param {type:"boolean"}
settings_file = "custom" #@param ["custom", "512x512_aesthetic_0.json","512x512_aesthetic_1.json","512x512_colormatch_0.json","512x512_colormatch_1.json","512x512_colormatch_2.json","512x512_colormatch_3.json"]
custom_settings_file = "/content/drive/MyDrive/Settings.txt"#@param {type:"string"}

def DeforumArgs():
    #@markdown **Image Settings**
    W = 720 #@param
    H = 405 #@param
    W, H = map(lambda x: x - x % 64, (W, H))  # resize to integer multiple of 64
    bit_depth_output = 8 #@param [8, 16, 32] {type:"raw"}

    #@markdown **Sampling Settings**
    seed = -1 #@param
    sampler = 'dpmpp_2s_a' #@param ["klms","dpm2","dpm2_ancestral","heun","euler","euler_ancestral","plms", "ddim", "dpm_fast", "dpm_adaptive", "dpmpp_2s_a", "dpmpp_2m"]
    steps = 50 #@param
    scale = 8 #@param
    ddim_eta = 0.0 #@param
    dynamic_threshold = None
    static_threshold = None   

    #@markdown **Save & Display Settings**
    save_samples = True #@param {type:"boolean"}
    save_settings = True #@param {type:"boolean"}
    display_samples = True #@param {type:"boolean"}
    save_sample_per_step = False #@param {type:"boolean"}
    show_sample_per_step = False #@param {type:"boolean"}

    #@markdown **Prompt Settings**
    prompt_weighting = True #@param {type:"boolean"}
    normalize_prompt_weights = True #@param {type:"boolean"}
    log_weighted_subprompts = False #@param {type:"boolean"}

    #@markdown **Batch Settings**
    n_batch = 1 #@param
    batch_name = "StableFun" #@param {type:"string"}
    filename_format = "{timestring}_{index}_{prompt}.png" #@param ["{timestring}_{index}_{seed}.png","{timestring}_{index}_{prompt}.png"]
    seed_behavior = "iter" #@param ["iter","fixed","random","ladder","alternate"]
    seed_iter_N = 1 #@param {type:'integer'}
    make_grid = False #@param {type:"boolean"}
    grid_rows = 2 #@param 
    outdir = get_output_folder(root.output_path, batch_name)

    #@markdown **Init Settings**
    use_init = False #@param {type:"boolean"}
    strength = 0.1 #@param {type:"number"}
    strength_0_no_init = True # Set the strength to 0 automatically when no init image is used
    init_image = "https://cdn.pixabay.com/photo/2022/07/30/13/10/green-longhorn-beetle-7353749_1280.jpg" #@param {type:"string"}
    # Whiter areas of the mask are areas that change more
    use_mask = False #@param {type:"boolean"}
    use_alpha_as_mask = False # use the alpha channel of the init image as the mask
    mask_file = "https://www.filterforge.com/wiki/images/archive/b/b7/20080927223728%21Polygonal_gradient_thumb.jpg" #@param {type:"string"}
    invert_mask = False #@param {type:"boolean"}
    # Adjust mask image, 1.0 is no adjustment. Should be positive numbers.
    mask_brightness_adjust = 1.0  #@param {type:"number"}
    mask_contrast_adjust = 1.0  #@param {type:"number"}
    # Overlay the masked image at the end of the generation so it does not get degraded by encoding and decoding
    overlay_mask = True  # {type:"boolean"}
    # Blur edges of final overlay mask, if used. Minimum = 0 (no blur)
    mask_overlay_blur = 5 # {type:"number"}

    #@markdown **Exposure/Contrast Conditional Settings**
    mean_scale = 0 #@param {type:"number"}
    var_scale = 0 #@param {type:"number"}
    exposure_scale = 0 #@param {type:"number"}
    exposure_target = 0.5 #@param {type:"number"}

    #@markdown **Color Match Conditional Settings**
    colormatch_scale = 0 #@param {type:"number"}
    colormatch_image = "https://www.saasdesign.io/wp-content/uploads/2021/02/palette-3-min-980x588.png" #@param {type:"string"}
    colormatch_n_colors = 4 #@param {type:"number"}
    ignore_sat_weight = 0 #@param {type:"number"}

    #@markdown **CLIP\Aesthetics Conditional Settings**
    clip_name = 'ViT-L/14' #@param ['ViT-L/14', 'ViT-L/14@336px', 'ViT-B/16', 'ViT-B/32']
    clip_scale = 0 #@param {type:"number"}
    aesthetics_scale = 0 #@param {type:"number"}
    cutn = 1 #@param {type:"number"}
    cut_pow = 0.0001 #@param {type:"number"}

    #@markdown **Other Conditional Settings**
    init_mse_scale = 0 #@param {type:"number"}
    init_mse_image = "https://cdn.pixabay.com/photo/2022/07/30/13/10/green-longhorn-beetle-7353749_1280.jpg" #@param {type:"string"}

    blue_scale = 0 #@param {type:"number"}
    
    #@markdown **Conditional Gradient Settings**
    gradient_wrt = 'x0_pred' #@param ["x", "x0_pred"]
    gradient_add_to = 'both' #@param ["cond", "uncond", "both"]
    decode_method = 'linear' #@param ["autoencoder","linear"]
    grad_threshold_type = 'dynamic' #@param ["dynamic", "static", "mean", "schedule"]
    clamp_grad_threshold = 0.2 #@param {type:"number"}
    clamp_start = 0.2 #@param
    clamp_stop = 0.01 #@param
    grad_inject_timing = list(range(1,10)) #@param

    #@markdown **Speed vs VRAM Settings**
    cond_uncond_sync = True #@param {type:"boolean"}

    n_samples = 1 # doesnt do anything
    precision = 'autocast' 
    C = 4
    f = 8

    prompt = ""
    timestring = ""
    init_latent = None
    init_sample = None
    init_sample_raw = None
    mask_sample = None
    init_c = None
    seed_internal = 0

    return locals()

args_dict = DeforumArgs()
anim_args_dict = DeforumAnimArgs()

if override_settings_with_file:
    load_args(args_dict, anim_args_dict, settings_file, custom_settings_file, verbose=False)

args = SimpleNamespace(**args_dict)
anim_args = SimpleNamespace(**anim_args_dict)

args.timestring = time.strftime('%Y%m%d%H%M%S')
args.strength = max(0.0, min(1.0, args.strength))

# Load clip model if using clip guidance
if (args.clip_scale > 0) or (args.aesthetics_scale > 0):
    root.clip_model = clip.load(args.clip_name, jit=False)[0].eval().requires_grad_(False).to(root.device)
    if (args.aesthetics_scale > 0):
        root.aesthetics_model = load_aesthetics_model(args, root)

if args.seed == -1:
    args.seed = random.randint(0, 2**32 - 1)
if not args.use_init:
    args.init_image = None
if args.sampler == 'plms' and (args.use_init or anim_args.animation_mode != 'None'):
    print(f"Init images aren't supported with PLMS yet, switching to KLMS")
    args.sampler = 'klms'
if args.sampler != 'ddim':
    args.ddim_eta = 0

if anim_args.animation_mode == 'None':
    anim_args.max_frames = 1
elif anim_args.animation_mode == 'Video Input':
    args.use_init = True

# clean up unused memory
gc.collect()
torch.cuda.empty_cache()

# dispatch to appropriate renderer
if anim_args.animation_mode == '2D' or anim_args.animation_mode == '3D':
    render_animation(args, anim_args, animation_prompts, root)
elif anim_args.animation_mode == 'Video Input':
    render_input_video(args, anim_args, animation_prompts, root)
elif anim_args.animation_mode == 'Interpolation':
    render_interpolation(args, anim_args, animation_prompts, root)
else:
    render_image_batch(args, prompts, root)

# Create Video From Frames

In [ ]:
skip_video_for_run_all = False #@param {type: 'boolean'}
fps = 24 #@param {type:"number"}
#@markdown **Manual Settings**
use_manual_settings = False #@param {type:"boolean"}
image_path = "/content/drive/MyDrive/AI/StableDiffusion/2023-01/StableFun/20230101212135_%05d.png" #@param {type:"string"}
mp4_path = "/content/drive/MyDrive/AI/StableDiffusion/2023-01/StableFun/output_video.mp4" #@param {type:"string"}
render_steps = False  #@param {type: 'boolean'}
path_name_modifier = "x0_pred" #@param ["x0_pred","x"]
make_gif = False
bitdepth_extension = "exr" if args.bit_depth_output == 32 else "png"

if skip_video_for_run_all == True:
    print('Skipping video creation, uncheck skip_video_for_run_all if you want to run it')
else:
    import os
    import subprocess
    from base64 import b64encode

    print(f"{image_path} -> {mp4_path}")

    if use_manual_settings:
        max_frames = "719" #@param {type:"string"}
    else:
        if render_steps: # render steps from a single image
            fname = f"{path_name_modifier}_%05d.png"
            all_step_dirs = [os.path.join(args.outdir, d) for d in os.listdir(args.outdir) if os.path.isdir(os.path.join(args.outdir,d))]
            newest_dir = max(all_step_dirs, key=os.path.getmtime)
            image_path = os.path.join(newest_dir, fname)
            print(f"Reading images from {image_path}")
            mp4_path = os.path.join(newest_dir, f"{args.timestring}_{path_name_modifier}.mp4")
            max_frames = str(args.steps)
        else: # render images for a video
            image_path = os.path.join(args.outdir, f"{args.timestring}_%05d.{bitdepth_extension}")
            mp4_path = os.path.join(args.outdir, f"{args.timestring}.mp4")
            max_frames = str(anim_args.max_frames)

    # make video
    cmd = [
        'ffmpeg',
        '-y',
        '-vcodec', bitdepth_extension,
        '-r', str(fps),
        '-start_number', str(0),
        '-i', image_path,
        '-frames:v', max_frames,
        '-c:v', 'libx264',
        '-vf',
        f'fps={fps}',
        '-pix_fmt', 'yuv420p',
        '-crf', '17',
        '-preset', 'veryfast',
        '-pattern_type', 'sequence',
        mp4_path
    ]
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    if process.returncode != 0:
        print(stderr)
        raise RuntimeError(stderr)

    mp4 = open(mp4_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display.display(display.HTML(f'<video controls loop><source src="{data_url}" type="video/mp4"></video>') )
    
    if make_gif:
         gif_path = os.path.splitext(mp4_path)[0]+'.gif'
         cmd_gif = [
             'ffmpeg',
             '-y',
             '-i', mp4_path,
             '-r', str(fps),
             gif_path
         ]
         process_gif = subprocess.Popen(cmd_gif, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [ ]:
skip_disconnect_for_run_all = True #@param {type: 'boolean'}

if skip_disconnect_for_run_all == True:
    print('Skipping disconnect, uncheck skip_disconnect_for_run_all if you want to run it')
else:
    from google.colab import runtime
    runtime.unassign()